# part A - .expand_as():

In this section, we implemented the my_expand_as function, which manually simulates PyTorch’s expand_as functionality without using built-in broadcasting functions.

The function:
*   Adds leading dimensions of size 1 to match the shape of the target tensor B
*   Iterates through each dimension and checks if broadcasting is possible
*   If a dimension in A is 1 and needs to match a larger size in B, it duplicates the data using select and stack
*   If broadcasting is not possible (i.e., incompatible shapes), it raises a ValueError

In [ ]:
import torch

def my_expand_as(A, B):
    shape_a = list(A.shape) # Dimensions of A.
    shape_b = list(B.shape) # Dimensions of B.

    # Adding dimensions to A according to B.
    while len(shape_a) < len(shape_b):
        A = A.unsqueeze(0)
        shape_a = list(A.shape)

    # Checking whether it is possible to broadcast.
    output = A.clone() # Duplication the Tensor.
    for i in range(len(shape_b)):
        dim_a = output.shape[i]
        dim_b = shape_b[i]

        if dim_a == dim_b: # The sizes of the dimensions are equal.
            continue  #  There is no need to broadcast.
        elif dim_a == 1: # The sizes of the dimensions are not equal, But the size of dim A is 1.
            slices = [output.select(i, 0) for _ in range(dim_b)] # Create copies of the single element in dimension i.
            output = torch.stack(slices, dim=i) # Rebuild tensor with expanded dimension i using the copies

        else:
            raise ValueError(f"Cannot broadcast dimension {dim_a} to {dim_b}")

    return output

In [ ]:
# Basic function validation check
def check_my_func(C, C_2):
    if torch.equal(C, C_2):
        print("True")
    else:
        print("False")

# --- Test 1 ---
A = torch.tensor([[1], [2]])        # shape: [2,1]
B = torch.zeros(2, 3)               # shape: [2,3]
C = my_expand_as(A, B)
C_2 = A.expand_as(B)
check_my_func(C, C_2)

# --- Test 2 ---
A = torch.tensor([[[1]]])           # shape: [1,1,1]
B = torch.zeros(2, 3, 4)            # shape: [2,3,4]
C = my_expand_as(A, B)
C_2 = A.expand_as(B)
check_my_func(C, C_2)

True
True


# Part B - is_broadcastable:

In this section, we implement the is_broadcastable function, which checks whether two tensors A and B can be broadcast together according to PyTorch's broadcasting rules.

Instead of modifying the tensors themselves, we only work with their shapes (A.shape and B.shape).
We align the shapes by prepending dimensions of size 1 when necessary, and check if each dimension pair is either equal or one of them is 1.

The function returns:
*   True and the resulting broadcasted shape (as a tuple) if broadcasting is possible.
*   False and None if the shapes are not compatible for broadcasting.

In [ ]:
def is_broadcastable(A, B):
    shape_a = list(A.shape)  # Get shape of A.
    shape_b = list(B.shape)  # Get shape of B.

    # Add 1 as need to the shorter shape so both shapes have the same length.
    while len(shape_a) < len(shape_b):
        shape_a = [1] + shape_a
    while len(shape_b) < len(shape_a):
        shape_b = [1] + shape_b

    result_shape = []  # Will hold the final broadcasted shape.

    # Check each dimension if its broadcastable.
    for dim_a, dim_b in zip(shape_a, shape_b):
        if dim_a == dim_b:
            result_shape.append(dim_a)  # Same size.
        elif dim_a == 1:
            result_shape.append(dim_b)  # A can be broadcast to B.
        elif dim_b == 1:
            result_shape.append(dim_a)  # B can be broadcast to A.
        else:
            return False, None  # Cannot broadcast

    return True, tuple(result_shape)  # Return success and the broadcasted shape

In [ ]:
# Basic function validation check
A = torch.tensor([[1], [2]])         # shape: [2, 1]
B = torch.tensor([[10, 20, 30]])     # shape: [1, 3]
print(is_broadcastable(A, B))

(True, (2, 3))


# Part C - .torch.broadcast_tensors:

This function performs addition between two tensors.
It uses the functions is_broadcastable and my_expand_as to reshape both tensors manually before performing the addition.


In [ ]:
def my_broadcast(A, B):
    # Check if tensors can be broadcasted.
    can_broadcast, target_shape = is_broadcastable(A, B)
    if not can_broadcast: # If can't broadcast.
        raise ValueError("Tensors cannot be broadcasted")

    # Expand A and B to the target shape.
    A_exp = my_expand_as(A, torch.empty(target_shape))
    B_exp = my_expand_as(B, torch.empty(target_shape))

    # Return both expanded tensors
    return A_exp, B_exp

# Part D - test:

In this section, we test our custom implementation of broadcast_tensors (my_broadcast)
by comparing it with PyTorch's built-in function torch.broadcast_tensors.

Each test checks whether the two input tensors are broadcast correctly,
and whether the output tensors match PyTorch's results.

In [ ]:
# Function to compare my manual broadcasting implementation with PyTorch's broadcast_tensors
def check_broadcast_tensors(A, B):
    try:
        # Try to apply PyTorch's built-in broadcasting
        A_expected, B_expected = torch.broadcast_tensors(A, B)
        expected_success = True  # Mark that broadcasting succeeded
    except RuntimeError as e:
        # PyTorch failed to broadcast the tensors
        expected_success = False
        expected_error = str(e)  # Save the error message for comparison

    try:
        # Try to apply my manual broadcasting implementation
        A_result, B_result = my_broadcast(A, B)
        result_success = True  # Mark that broadcasting succeeded
    except Exception as e:
        # My function failed to broadcast the tensors
        result_success = False
        result_error = str(e)  # Save the error message for comparison

    # Compare the results of both implementations
    if expected_success and result_success:
        # Both succeeded – now compare the actual broadcasted tensors
        same_A = torch.equal(A_expected, A_result)
        same_B = torch.equal(B_expected, B_result)
        print(f"✅ Both succeeded.\n\tA match: {same_A} \n\tB match: {same_B}")
    elif not expected_success and not result_success:
        # Both implementations raised an error – that's correct
        print("✅ Both raised an error as expected")
    else:
        # One succeeded while the other failed – mismatch
        print("❌ The results are not the same")
        if expected_success:
            print(f"Expected success, got error: {result_error}")
        else:
            print(f"Expected error: {expected_error}, but function returned a result.")


In [ ]:
# Test cases

print("Test 1: [2,1] and [1,3]")
check_broadcast_tensors(torch.tensor([[1], [2]]), torch.tensor([[10, 20, 30]]))

print("\nTest 2: [3] and [2,1]")
check_broadcast_tensors(torch.tensor([1, 2, 3]), torch.tensor([[1], [2]]))

print("\nTest 3: [1,1,1] and [2,3,4]")
check_broadcast_tensors(torch.tensor([[[1]]]), torch.empty(2, 3, 4))

print("\nTest 4: Incompatible shapes [2] and [3]")
check_broadcast_tensors(torch.tensor([2, 3]), torch.tensor([3, 4]))

print("\nTest 5: [1,3] and [3]")
check_broadcast_tensors(torch.tensor([[1, 2, 3]]), torch.tensor([10, 20, 30]))

print("\nTest 6: Incompatible shapes [2] and [3]")
check_broadcast_tensors(torch.tensor([1, 2]), torch.tensor([1, 2, 3]))

Test 1: [2,1] and [1,3]
✅ Both succeeded.
	A match: True 
	B match: True

Test 2: [3] and [2,1]
✅ Both succeeded.
	A match: True 
	B match: True

Test 3: [1,1,1] and [2,3,4]
✅ Both succeeded.
	A match: True 
	B match: True

Test 4: Incompatible shapes [2] and [3]
✅ Both succeeded.
	A match: True 
	B match: True

Test 5: [1,3] and [3]
✅ Both succeeded.
	A match: True 
	B match: True

Test 6: Incompatible shapes [2] and [3]
✅ Both raised an error as expected


# Summary:

In this assignment, we manually implemented key broadcasting operations in PyTorch without using any built-in broadcasting functions such as expand, broadcast_to, or broadcast_tensors.

The main goals of the assignment were:

*   Implement my_expand_as(A, B) to manually expand a tensor to match a target shape
*   Implement is_broadcastable(A, B) to check whether two tensors can be broadcast together and return the resulting shape
*   Implement my_broadcast(A, B) to simulate torch.broadcast_tensors, returning both broadcast tensors
*   Compare the custom implementations with PyTorch's behavior through a set of test cases, including both valid and invalid scenarios

This exercise deepened our understanding of how broadcasting works under the hood and how to manually manipulate tensor shapes using operations like `unsqueeze`, `stack`, and `select`.